In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title cd
%pwd
%cd /content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet

/content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet


In [3]:
#@title install packages
%pip install chainer
%pip install -Uqq ipdb

     |████████████████████████████████| 792 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 374 kB 86.2 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.24 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.1 which is incompatible.


In [4]:
#@title read gpu details
import chainer
#chainer.print_runtime_info()

print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Wed Jan 26 12:49:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
#@title imports
"""
high level support for doing this and that.
"""
from __future__ import print_function
import time
import csv
import numpy as np
import torch
import torch.nn.functional as F
from scipy import stats
from init import initializer
from eenet import EENet
from custom_eenet import CustomEENet
from gatednet import GatedNet
import matplotlib.pylab as plt
import loss_functions
import utils
import config
import gc
import sys
import math
import os
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms.functional as tf
import matplotlib.pyplot as plt
import ipdb

#import xgboost as xgb
#from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import accuracy_score
#import joblib as jobl
from collections import Counter
from matplotlib import pyplot
from numpy import where
#rom imblearn.over_sampling import RandomOverSampler 
#from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#from xgboost.sklearn import XGBRegressor

%matplotlib inline


In [6]:
#@title plot_ee_histogram
def plot_ee_histogram(args, val_confs, type_loader, dir):
    %matplotlib inline
    x_pos = np.arange(args.num_ee+1)
    #print(x_pos)
    print(val_confs)
    bar = plt.bar(x_pos, val_confs, align='center')
    plt.xlabel('Exit number')
    plt.ylabel('# of Samples in Exit')
    plt.title(args.dataset + ' ' +  type_loader + ' samples distribution in exits using ' + args.model + 
              ' model\n')
    plt.ylim(0, max(val_confs))
    plt.savefig(dir + '/ee_hist_' + type_loader + '.' + str(args.loss_threshold) + '.png')
    plt.show()
    

In [7]:
#@title generate_ee_exit_prediction_ground_truths

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def test_gated(args, main_model, gated_model, test_loader):
    
    main_model.eval()
    gated_model.eval()
    target_match_count = 0
    accs_main = []
    accs_gated = []
            
    with torch.no_grad():
        for data, target_org in test_loader:
            
            batch_size = target_org.shape[0]        
            data, target_org = data.to(args.device), target_org.to(args.device) 
            preds_main, _ = main_model(data)
            d1, d2, d3, d4 = data.shape
            flat_data = data.reshape((d1, d2*d3*d4))    
            #preds_gated = gated_model(flat_data)
            preds_gated = gated_model(data)

            if args.use_main_targets:
                _, target = torch.max(preds_main[args.num_ee], 1) 
            
            #ipdb.set_trace(context=6)
            
            pred_losses = torch.empty((args.num_ee + 1, batch_size), dtype=torch.float)
            pred_main = torch.zeros((batch_size, preds_main[0].shape[1]), dtype=torch.float, device = args.device)
            pred_gated = torch.zeros((batch_size, preds_main[0].shape[1]), dtype=torch.float, device = args.device)
            best_ee_target_batch = np.full(batch_size, args.num_ee, dtype=int)
            
            for i in range(args.num_ee+1):
                #cost = model.complexity[i][0]/model.complexity[-1][0]
                pred_losses[i] = F.nll_loss(preds_main[i].log(), target, reduction='none')
            #main_ee_loss = F.nll_loss(preds_main[args.num_ee].log(), target)
            #main_ee_losses.append(float(main_ee_loss))
            gated_target_pred = torch.max(preds_gated, 1).indices
            #ipdb.set_trace(context=6)
            
            for j in range(batch_size):
                best_exit = args.num_ee
                for i in range(args.num_ee):    
                    if pred_losses[i][j] < args.loss_threshold:
                        best_exit = i
                        break
                
                #ipdb.set_trace(context=6)
                best_ee_target_batch[j] = best_exit
                pred_main[j] = preds_main[best_exit][j]
                #best_exit_gated_target_pred = torch.max(preds_gated[j], 1).indices
                pred_gated[j] = preds_main[gated_target_pred[j]][j]
                
                #best_ee_target_batch[j] = best_ee_target_pred
                
            acc_main = accuracy(pred_main.float().data, target_org)[0]
            acc_gated = accuracy(pred_gated.float().data, target_org)[0]
            #print('acc main: ', acc_main)
            #print('acc gated: ', acc_gated)
            pred = None
            
            accs_main.append(float(acc_main))
            accs_gated.append(float(acc_gated))
            #ipdb.set_trace(context=6)
            
            #log_file.write("Accuracy: " + str(accu) + "\n")
            #print('Batch Accuracy main: {:.4f}'.format(float(acc_main)))
            #print('Batch Accuracy gated: {:.4f}'.format(float(acc_gated)))
        
        print('Total avg Accuracy main: {:.4f}'.format(round(np.mean(accs_main), 4)))
        print('Total avg Accuracy gated: {:.4f}'.format(round(np.mean(accs_gated), 4)))
            
            
            

In [8]:
#@title load data
from torch.utils.data import Dataset, TensorDataset, DataLoader

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        #ipdb.set_trace(context=6)
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]
        
        return x, y

    def __len__(self):
        return self.tensors[0].size(0)
        
#def load_data(args):
#    return utils.load_custom_dataset(args)

def load_data(args):
    serialized_data_dir = 'serialized_data'
    
    data_file_test = serialized_data_dir + '/datas_test' + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    
    gt_file_test = serialized_data_dir + '/targets_test' + '.pt'
    target_test = torch.load(gt_file_test)
    print('loaded exits ground truths test at: ', gt_file_test)

    data_test, target_test = torch.from_numpy(data_test), torch.from_numpy(target_test)
    
    test_set = CustomTensorDataset(tensors=(data_test, target_test))
    loader_test = torch.utils.data.DataLoader(test_set, batch_size=args.test_batch, shuffle=args.shuffle_test)

    return loader_test

In [9]:
#@title main

"""Main function of the program.

The function loads the dataset and calls training and validation functions.
"""
%load_ext autoreload
%autoreload 2

import importlib
importlib.reload(config)
importlib.reload(loss_functions)
importlib.reload(utils)
from enum import Enum

class Mode(Enum):
    train_main = 0
    train_ee = 1
    generate_exits_gt = 2 
    train_gating = 3
    generate_relative_loss = 4 
    plot_relative_loss = 5
    test_gated = 6

def main(mode: Mode):
    print(mode.value)
    args = config.args_global
    args += config.argu[mode.value]
    
    %pwd
    main_model, optimizer, lr_scheduler, args = initializer(args)
    print(args)
    
    if mode == Mode.test_gated:
        #train_loader, test_loader, exit_tags, trainset, testset = load_data(args)       
        test_loader = load_data(args)       
        for thresh in range(2, 22, 3):
            args.loss_threshold = thresh/10
            gaiting_model_file = 'gated_models/' + str(args.dataset) + '/model.' + str(args.loss_threshold) + '.pt'
            print('load gated model: ', gaiting_model_file)
            gated_model = torch.load(gaiting_model_file)
            print('Test gated framework, threshold = {:.2f}'.format(args.loss_threshold))
            test_gated(args, main_model, gated_model, test_loader)
    
    print('Finished')

In [10]:
mode = Mode.test_gated
main(mode)

6
use cuda:  True  device:  cuda
empty model loaded at:  main_models/models/cifar10/eenet110/UT/clean/after_ee_training/model.pt
ee-block-0: flops=19.78 MMac, params=19.34 k, cost-rate=0.08
ee-block-1: flops=34.23 MMac, params=33.35 k, cost-rate=0.13
ee-block-2: flops=43.86 MMac, params=42.7 k, cost-rate=0.17
ee-block-3: flops=53.49 MMac, params=52.04 k, cost-rate=0.21
ee-block-4: flops=63.13 MMac, params=61.39 k, cost-rate=0.25
ee-block-5: flops=72.76 MMac, params=70.73 k, cost-rate=0.28
ee-block-6: flops=82.4 MMac, params=80.08 k, cost-rate=0.32
ee-block-7: flops=92.03 MMac, params=89.42 k, cost-rate=0.36
ee-block-8: flops=100.48 MMac, params=136.57 k, cost-rate=0.39
ee-block-9: flops=110.01 MMac, params=173.69 k, cost-rate=0.43
exit-block: flops=256.32 MMac, params=1.73 M, cost-rate=1.00
Namespace(adaptive_lr=False, add_noise=False, batch_size=32, clear_dirs=False, dataset='cifar10', device=device(type='cuda'), distribution='fine', ee_costs=None, epochs=150, exit_type='conv2', filte

FileNotFoundError: [Errno 2] No such file or directory: 'gated_models/cifar10/model.0.5.pt'